# Vampire Werewolves strategy training

In [56]:
import numpy as np
import random
import copy

In [67]:
class Size:
    """
    A size object for 2D boards, with x and y as attributes
    """
    def __init__(self, width, height):
        self.x = width
        self.y = height
        

class Board:
    """
    A game board for the game. 
     ------------->  Y
    |
    |
    |
    |
    |
    v
    
    X
    
    3rd dimension used for number of Vampires, Werewolves, Humans
    
    params:
    width (size of the board)
    height (size of the board)
    
    attributes:
    board
        1st dimension : x
        2nd dimension : y
        3rd dimension : [number_of_vampires, number_of_werewolves, number_of_humans]
    
    """
    
    # Ids of Vampires, Werewolves and Humans in the 3rd board dimension
    id_V = 0
    id_W = 1
    id_H = 2
        
    def __init__(self, width, height):
        """
        Init board with given width and height
        """
        
        self.size = Size(width, height)
        self.board = np.zeros((self.size.x, self.size.y, 3), dtype=np.int8)
        self.init_board()
    
    
    def init_board(self):
        """
        Init the self.board by creating vampires, werewolves and humans in it
        """
        
        ## Affecting vampires in board
        n_vampires = 10
        # Creating a list of possible position in board. We positionate vampires first, every position is possible
        potential_vampires_position = [(x,y) for x in range(self.size.x) for y in range(self.size.y)]
        vampires_position = random.choice(potential_vampires_position)
        x_vampires, y_vampires = vampires_position
        self.board[x_vampires, y_vampires, self.id_V] = n_vampires
            
        ## Affecting werewolves
        n_werewolves = 10
        # Creating a list of possible position in board. We remove vampires position from possible ones
        potential_werewolves_position = [(x,y) for x in range(self.size.x) for y in range(self.size.y)]
        potential_werewolves_position.remove(vampires_position)
        werewolves_position = random.choice(potential_werewolves_position)
        x_werewolves, y_werewolves = werewolves_position
        self.board[x_werewolves, y_werewolves, self.id_W] = n_werewolves
        
        ## Affecting humans
        n_humans = [3, 3, 5]
        humans_position = []
        
        for group_n_humans in n_humans:
            # Creating a list of possible position in board. We remove vampires, werewolves, and other humans positions.
            potential_humans_position = [(x,y) for x in range(self.size.x) for y in range(self.size.y)]
            potential_humans_position.remove(vampires_position)
            potential_humans_position.remove(werewolves_position)
            for previous_human_position in humans_position:
                potential_humans_position.remove(previous_human_position)
        
            group_humans_position = random.choice(potential_humans_position)
            # Storing this group position
            humans_position.append(group_humans_position)
            x_group_humans, y_group_humans = group_humans_position
            self.board[x_group_humans, y_group_humans, self.id_H] = group_n_humans
    
    def display(self):
        """
        Display the board in a ergonomic way, with number of Vampires, Werewolves and Humans on cells.
        They cannot be 2 species in the same cell
        """
        
        for x in range(self.size.x):
            line = ""
            for y in range(self.size.y):
                line += "|"
                if self.board[x,y,self.id_V] != 0:
                    cell = str(int(self.board[x,y,self.id_V])) + "V"
                    while len(cell) < 3:
                        cell += " "
                    line += cell
                elif self.board[x,y,self.id_W] != 0:
                    cell = str(int(self.board[x,y,self.id_W])) + "W"
                    while len(cell) < 3:
                        cell += " "
                    line += cell
                elif self.board[x,y,self.id_H] != 0:
                    cell = str(int(self.board[x,y,self.id_H])) + "H"
                    while len(cell) < 3:
                        cell += " "
                    line += cell
                else:
                    line += "   "
            
            line += "|"
            print("-"*len(line))
            print(line)
        print("-"*len(line))
                
# Creating a 5*5 board and display it
game_board = Board(4, 4)
game_board.display()

-----------------
|   |   |   |   |
-----------------
|   |   |   |   |
-----------------
|3H |5H |10W|   |
-----------------
|3H |10V|   |   |
-----------------


In [76]:
"""TODO:
- Create a way to generate a tree of possible moves and use min-max and 
alpha-beta techniques to explore this tree and select best move

"""

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return str(self.x) + "_" + str(self.y)


class Player:
    def __init__(self, name, species):
        # Storing player name
        self.name = name
        
        # Storing player species
        if species == "V":
            print(self.name, "is playing vampires")
            self.species = species
            self.id_species = Board.id_V
        elif species == "W":
            print(self.name, "is playing werewolves")
            self.species = species
            self.id_species = Board.id_W
        else:
            raise ValueError("Don't know the species " + str(species))
        
    def possible_move(self, game_board_):
        """
        :param game_board: the game board object
        
        Return a list of possible moves in this format :
        (original_point, number_of_creature_moving, final_point, score_of_move, new_potential_board)
        """
        self.game_board = game_board_
        
        # Scanning where are our creatures
        our_creatures_groups_position_raw = np.argwhere(self.game_board.board[:,:,self.id_species] != 0)
        
        # Storing their position as a Point object and storing the number of creatures in each group
        our_creatures_groups_position = []
        our_creatures_groups_population = []
        for group_position_raw in our_creatures_groups_position_raw:
            group_position = Point(group_position_raw[0], group_position_raw[1])
            our_creatures_groups_position.append(group_position)
            our_creatures_groups_population.append(self.game_board.board[group_position.x, group_position.y, self.id_species])
        
        # NOTE : for now, we will not consider splitting the creatures group
        
        # Exploring possible moves for each group
        moves = []
        for id_group, group_position in enumerate(our_creatures_groups_position):
            for delta_x in [-1, 0, 1]:
                for delta_y in [-1, 0, 1]:
                    new_x = group_position.x + delta_x
                    new_y = group_position.y + delta_y
                    if new_x < 0 or new_x >= self.game_board.size.x or new_y < 0 or new_y >= self.game_board.size.y or (delta_x == 0 and delta_y == 0):
                        # The new point is out of the game board
                        continue
                    else:
                        new_potential_position = Point(new_x, new_y)
                        group_population = our_creatures_groups_population[id_group]
                        score, new_board = self.score_move(group_position, group_population, new_potential_position)
                        if score == -666:
                            # We are not allowing this move
                            continue
                        moves.append([group_position, group_population, new_potential_position, score, new_board])
        return moves
            
    
    def score_move(self, origin_position, our_creature_population, target_position):
        """
        Return the score of the proposed creatures move along with the new board related to this move
        
        :return score: 
            -666: if the move is forbidden
            <0: if the move makes us loss creatures more than killing others for instance 
            0: if the move is neutral
            >0: if we have converted humans, or kill other creatures more than they killed us
        :return new_board: 
            new board state
        """
        
        target_cell = self.game_board.board[target_position.x, target_position.y]

        if max(target_cell) == 0:
            # There are no species in this cell, then the score is neutral
            score = 0
            ## Lets build the new board of this potentality
            new_game_board = copy.deepcopy(self.game_board)
            # We are leaving the original cell
            new_game_board.board[origin_position.x, origin_position.y, self.id_species] = 0
            # We move to the new cell
            new_game_board.board[target_position.x, target_position.y, self.id_species] = our_creature_population
            return score, new_game_board
            
        elif target_cell[self.id_species] != 0:
            # We are merging with our own species
            print(target_cell)
            raise NotImplementedError("Error, we haven't considered splitting, neither merging, groups yet")
        
        elif target_cell[self.game_board.id_H] != 0:
            # We are meeting humans
            number_of_humans = target_cell[self.game_board.id_H]
            if our_creature_population > number_of_humans*1.5:
                # If we are 50% more than humans, we convert all of them
                # Lets consider this as a score equal to "number of converted humans"
                score = number_of_humans
                
                ## Lets build the new board of this potentality
                new_game_board = copy.deepcopy(self.game_board)
                # We are leaving the original cell
                new_game_board.board[origin_position.x, origin_position.y, self.id_species] = 0
                # We are removing humans from the targeted cell
                new_game_board.board[target_position.x, target_position.y, self.game_board.id_H] = 0
                # We sum number of converted humans and previous our_creatures to create new number of our_creatures
                new_game_board.board[target_position.x, target_position.y, self.id_species] = number_of_humans + our_creature_population
                return score, new_game_board
            else:
                # TODO: score if we are not enough to convert humans easily
                # For now, forbidding move
                return -666, None
        
        else:
            # If not humans and not our species, but still there is a species in this cell, it is the enemy creature
            enemy_id = (self.id_species + 1)%2
            number_of_enemy = target_cell[enemy_id]

            if our_creature_population > number_of_enemy*1.5:
                # We are killing every enemy
                # Lets consider this as a score equal to "number of converted humans" with a weight
                score = number_of_enemy
                
                ## Lets build the new board of this potentality
                new_game_board = copy.deepcopy(self.game_board)
                # We are leaving the original cell
                new_game_board.board[origin_position.x, origin_position.y, self.id_species] = 0
                # We are removing enemies from the targeted cell
                new_game_board.board[target_position.x, target_position.y, enemy_id] = 0
                # We move to the new cell
                new_game_board.board[target_position.x, target_position.y, self.id_species] = our_creature_population
                return score, new_game_board
            else:
                # TODO: score if we are not enough to convert humans easily
                # For now, forbidding move
                return -666, None
    
    def display_node(self, moves):
        for id_move, move in enumerate(moves):
            print("-"*10)
            print("Move n°", id_move)
            group_position, group_population, new_potential_position, score, new_board = move
            print("Moving", group_population, self.species, "from", group_position, "to", new_potential_position)
            print("Scored", score)
            print("New board:")
            new_board.display()
        
            
            
game_board.display()
player1 = Player("Dracula", "V")
moves = player1.possible_move(game_board)
player1.display_node(moves)
        

-----------------
|   |   |   |   |
-----------------
|   |   |   |   |
-----------------
|3H |5H |10W|   |
-----------------
|3H |10V|   |   |
-----------------
Dracula is playing vampires
----------
Move n° 0
Moving 10 V from 3_1 to 2_0
Scored 3
New board:
-----------------
|   |   |   |   |
-----------------
|   |   |   |   |
-----------------
|13V|5H |10W|   |
-----------------
|3H |   |   |   |
-----------------
----------
Move n° 1
Moving 10 V from 3_1 to 2_1
Scored 5
New board:
-----------------
|   |   |   |   |
-----------------
|   |   |   |   |
-----------------
|3H |15V|10W|   |
-----------------
|3H |   |   |   |
-----------------
----------
Move n° 2
Moving 10 V from 3_1 to 3_0
Scored 3
New board:
-----------------
|   |   |   |   |
-----------------
|   |   |   |   |
-----------------
|3H |5H |10W|   |
-----------------
|13V|   |   |   |
-----------------
----------
Move n° 3
Moving 10 V from 3_1 to 3_2
Scored 0
New board:
-----------------
|   |   |   |   |
------------